# Networks: structure, evolution & processes
**Internet Analytics - Lab 2**

---

**Group:** *P*

**Names:**

* *Matthias Leroy*
* *Pierre Fouche*
* *Alexandre Poussard*

---

#### Instructions

*This is a template for part 4 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

---

## 2.4 PageRank

### 2.4.1 Random Surfer Model

#### Exercise 2.12

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import collections

#Count the number of visit in a node
def ctr(node,graph):
    if graph.node[node]:
        graph.node[node] += 1
    else:
        graph.node[node] = 1

#Algorithm of the random surfer        
def randomSurfer(graph):
    #Choose random source
    node = np.random.randint(0,graph.number_of_nodes())
    node =str(node)
    ctr(node,graph)
    i = 0
    while graph.neighbors(node) and i<100:
        #Choose a rande neighbor
        tab = graph.neighbors(node)
        node = np.random.choice(tab)
        ctr(node,graph)
        i = i + 1
    print(graph.nodes(True))

In [ ]:
graph1 = nx.read_adjlist('../data/absorbing.graph', create_using = nx.DiGraph())
graph2 = nx.read_adjlist('../data/components.graph', create_using = nx.DiGraph())

randomSurfer(graph1)
randomSurfer(graph2)


We always observe the same pattern. It is because a graph can contain nodes with outdegree equal to zero or graph can have multiple connected components. And so the algorithm is stuck at a node or in a component.

#### Exercise 2.13

In [ ]:
#Choose random node
def randomNode(graph):
    r = np.random.randint(0,graph.number_of_nodes())
    ctr(str(r),graph)
    return str(r)
#Algorithm of the random surfer modified
def randomSurferBis(graph):
    #Random source
    node = randomNode(graph)
    N = 100
    for i in range(N):
        #Choose node at random with probability 0.15
        r = np.random.random()
        if(r <= 0.15):
            node = randomNode(graph)
        else:
            #Check if node has neighbors, if not choose random node
            if not graph.neighbors(node):
                node = randomNode(graph)
            else:
                tab = graph.neighbors(node)
                r = np.random.choice(tab)
                ctr(r,graph)
    print(graph.nodes(True))
    

In [ ]:
graph1 = nx.read_adjlist('../data/absorbing.graph', create_using = nx.DiGraph())
graph2 = nx.read_adjlist('../data/components.graph', create_using = nx.DiGraph())

randomSurferBis(graph1)
randomSurferBis(graph2)


---

### 2.4.2 Power Iteration Method

#### Exercise 2.14: Power Iteration method

In [ ]:
#Power iteration method
def powerIteration(G,pi):
    norm = 1
    #while the norm of the difference between new pi and old pi is too big we continue
    while norm > pow(10,-6):
        temp = pi
        pi = np.matmul(pi,G)
        norm = np.linalg.norm(pi-temp)
    return pi

#Print the top ten page of the page rank
def printTopTen(score,nbrNodes):
    dic = {}
    #Create dictionary with the top 10 nodes as keys
    for i in range(nbrNodes):
        dic[i] = score[0][i]
    sorted_node = sorted(dic.items(), key=lambda x: -x[1])
    topTen = sorted_node[:10]
    topTen = dict(topTen)
    result = dict.fromkeys(topTen.keys())
    
    #Search in wikipedia_titles the titles which correspond to the nodes
    with open("../data/wikipedia_titles.tsv") as title:
        for line in title:
            columns = line.split("\t")
            if columns[0] != "#page_id" and int(columns[0]) in list(topTen.keys()):
                result[int(columns[0])] = columns[1][:-1]
    print(result.values())

#Compute the page rank of each nodes    
def pageRank(graph):
    nbrNodes = graph.number_of_nodes()
    #Create matrix H
    H = np.zeros((nbrNodes,nbrNodes))
    for i in range(nbrNodes):
        for j in range(nbrNodes):
            if str(j) in graph.neighbors(str(i)):
                H[i][j] = 1/graph.out_degree(str(i))
    #Create vector w
    w = np.zeros((nbrNodes,1),int)
    ones = np.ones((1,nbrNodes),int)
    for i in range(nbrNodes):
        if not graph.neighbors(str(i)):
            w[i] = 1
    w = np.matmul(w,ones) * 1/nbrNodes
    H2 = H + w
    theta = 0.85
    temp = np.matmul(ones.transpose(),ones) * (1-theta) / nbrNodes
    G = theta * H2 + temp
    pi = np.ones((1,nbrNodes),int) / nbrNodes
    return powerIteration(G,pi)
    

    




In [ ]:
graph = nx.read_adjlist('../data/wikipedia.graph', create_using = nx.DiGraph())
printTopTen(pageRank(graph),graph.number_of_nodes())


The top ten pages are United Kingdom, Europe, England, France, World War II, Latin, Germany, English language, India and United States.

---

### 2.4.3 Gaming the system *(Bonus)*

#### Exercise 2.15 *(Bonus)*